<div dir="rtl">
  <center>
    <img src="https://user-images.githubusercontent.com/7662492/229289746-89c5a4c7-afa6-4d46-a0e6-63dfdeb98285.jpg" width="250px"/>

  <h1>تفريغ - الكتب المُيسّرة</h1>
  </center>

  <p>مرحبًا بك في تفريغ لتفريغ المواد الصوتية والمرئية باستخدام تقنيات الذكاء الاصطناعي. لاستخدام تفريغ:</p>

  <ol>
    <li>قم بإدخال روابط المواد المطلوب تحويلها من منصة YouTube أو أي منصة أخرى في حقل "urls" وتأكد من فصلها بمسافة، أو قم بترك الحقل فارغًا لتفريغ المواد التي قمت برفعها</li>
    <li>(اختياري) قم بتحديد أقل عدد من الكلمات في كل جزء من أجزاء التفريغ. يؤثر هذا في طول الأجزاء التي سيتم تفريغها</li>
    <li>
      إذا كنت تريد استخدام مجموعة نماذج Whisper:
      <ul>
        <li>قم باختيار النموذج المُراد استخدامه في حقل "model"</li>
        <li>قم باختيار لغة المادة في حقل "language"</li>
        <li>قم باختيار المهمة المُراد إنجازها في حقل "task" سواء كانت تفريغ المادة أو ترجمتها للانجليزية</li>
      </ul>
    </li>
    <li>
      إذا كنت تريد استخدام تقنية wit.ai:
      <ul>
        <li>قم بوضع مفتاح wit.ai الخاص بك في حقل "wit_api_key"</li>
        <li>(اختياري) قم بتحديد أقصى مدة للتقطيع والتي ستؤثر على طول الجمل في ملف المخرجات</li>
      </ul>
    </li>
    <li>قم بتشغيل الخلية في الأسفل من خلال الضغط على الدائرة المحتوية على السهم</li>
  </ol>

  <p>يمكنك تجربة التفريغ باستخدام نماذج Whisper وتقنية wit.ai واستخدام التفريغ الأفضل لحالتك. كملاحظة عامة، نماذج Whisper تقوم بتفريغ الهمزات والتشكيلات وعلامات الترقيم بشكل أفضل من wit.ai، لكن wit.ai يُنتج أخطاء إملائية أقل.</p>

  <p>عندما ينتهي التحويل سيتم تنزيل الملفات النصية بشكل تلقائي بصيغة <code>txt</code> و <code>srt</code> وسيكون اسم الملف هو مُعرّف المادة على منصة YouTube الذي يكون في آخر رابط المادة: https://youtu.be/<strong>4h5P7jXvW98</strong>.</p>

  <hr>

  <p>يمكنك متابعة مشروع <strong>الكتب المُيسّرة</strong> والتواصل معنا من خلال:</p>

  <ul>
    <li><a href="https://t.me/ieasybooks">قناتنا على تيليجرام</a></li>
    <li><a href="https://www.youtube.com/@ieasybooks">قناتنا على يوتيوب</a></li>
    <li><a href="https://twitter.com/iieasybooks">حسابنا على تويتر</a></li>
    <li><a href="https://www.facebook.com/ieasybooks">صفحتنا على فيسبوك</a></li>
    <li><a href="https://github.com/ieasybooks">حسابنا على GitHub (للمبرمجين)</a></li>
    <li>بريدنا الالكتروني: easybooksdev@gmail.com</li>
  </ul>
</div>

In [ ]:
# @title <h1><center>تفريغ</center></h1>

print('جارٍ تجهيز بيئة العمل.')

# Setup Tafrigh.
!apt-get install -q portaudio19-dev python3-pyaudio > pyaudio_fix_logs.txt
%pip install -U tafrigh[wit,whisper]==1.7.5 > install_logs.txt
%pip install -U yt-dlp > update_yt_dlp_logs.txt

# Start: Quick fix related to Colab, HuggingFace, and faster-whisper.
!apt install libcublas11 > fix_logs.txt
!pip install ctranslate2==4.4.0 > ctranslate2_downgrade_logs.txt

from huggingface_hub.utils import _runtime
_runtime._is_google_colab = False
# End: Quick fix related to Colab, HuggingFace, and faster-whisper.

# Get inputs.

# @markdown <h1 dir="rtl">مدخلات عامة</h1>

# @markdown <p dir="rtl">روابط المواد المطلوب تفريغها وتأكد من فصلها بمسافة، أو أترك الحقل فارغًا لتفريغ المواد التي قمت برفعها</p>
urls = 'https://youtu.be/4h5P7jXvW98 https://youtu.be/jpfndVSROpw'  # @param { type: "string" }

# @markdown <p dir="rtl">أقل عدد من الكلمات في كل جزء من أجزاء التفريغ</p>
min_words_per_segment = 1  # @param {type:"slider", min:1, max:100, step:1}

# @markdown ---

# @markdown <h1 dir="rtl">مدخلات خاصة بنماذج Whisper</h1>

# @markdown <p dir="rtl">النموذج المُراد استخدامه للتفريغ</p>
model = 'large-v3 (\u0623\u0641\u0636\u0644 \u062F\u0642\u0629)'  # @param ["large-v3 (أفضل دقة)", "medium", "base", "small", "tiny (أقل دقة)"]

# @markdown <p dir="rtl">(اختياري) لغة المادة</p>
language = 'ar'  # @param ["ar", "af", "am", "as", "az", "ba", "be", "bg", "bn", "bo", "br", "bs", "ca", "cs", "cy", "da", "de", "el", "en", "es", "et", "eu", "fa", "fi", "fo", "fr", "gl", "gu", "ha", "haw", "he", "hi", "hr", "ht", "hu", "hy", "id", "is", "it", "ja", "jw", "ka", "kk", "km", "kn", "ko", "la", "lb", "ln", "lo", "lt", "lv", "mg", "mi", "mk", "ml", "mn", "mr", "ms", "mt", "my", "ne", "nl", "nn", "no", "oc", "pa", "pl", "ps", "pt", "ro", "ru", "sa", "sd", "si", "sk", "sl", "sn", "so", "sq", "sr", "su", "sv", "sw", "ta", "te", "tg", "th", "tk", "tl", "tr", "tt", "uk", "ur", "uz", "vi", "yi", "yo", "zh"]

# @markdown <p dir="rtl">(اختياري) المهمة</p>
task = 'تفريغ'  # @param ["تفريغ", "ترجمة"]

# @markdown ---

# @markdown <h1 dir="rtl">مدخلات خاصة بتقنية wit.ai</h1>

# @markdown <p dir="rtl">المفتاح الخاص بك على موقع wit.ai</p>
wit_api_key = ''  # @param { type: "string" }

# @markdown <p dir="rtl">(اختياري) أقصى مدة للتقطيع والتي ستؤثر على طول الجمل في ملف SRT</p>
max_cutting_duration = 15  # @param {type:"slider", min:1, max:17, step:1}

if model == 'large-v3 (\u0623\u0641\u0636\u0644 \u062F\u0642\u0629)':
    model = 'large-v3'
elif model == 'tiny (\u0623\u0642\u0644 \u062F\u0642\u0629)':
    model = 'tiny'

if task == 'تفريغ':
  task = 'transcribe'
elif task == 'ترجمة':
  task = 'translate'

# Imports.
import glob
import os

from collections import deque

from google.colab import files
from tafrigh import farrigh, Config

# Setup directories.
output_dir = 'output'
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

# Start Tafrigh.
if wit_api_key:
  print('جارٍ تحويل المواد إلى نصوص باستخدام تقنيات wit.ai.')
else:
  print('جارٍ تحويل المواد إلى نصوص باستخدام نماذج Whisper.')

config = Config(
  input=Config.Input(
    urls_or_paths=list(map(str.strip, urls.split(' '))) if len(urls.strip()) else ['.'],
    skip_if_output_exist=False,
    download_retries=3,
    yt_dlp_options='{}',
    verbose=False,
  ),
  whisper=Config.Whisper(
    model_name_or_path=model,
    task=task,
    language=language,
    use_faster_whisper=True,
    beam_size=5,
    ct2_compute_type='default',
  ),
  wit=Config.Wit(
    wit_client_access_tokens=None if len(wit_api_key.strip()) == 0 else wit_api_key.split(),
    max_cutting_duration=max_cutting_duration,
  ),
  output=Config.Output(
    min_words_per_segment=min_words_per_segment,
    save_files_before_compact=False,
    save_yt_dlp_responses=False,
    output_sample=0,
    output_formats=['txt', 'srt'],
    output_dir=output_dir,
  ),
)

deque(farrigh(config), maxlen=0)

# Download all txt and srt files.
print('جارٍ تنزيل الملفات النصية.')

txt_files = glob.glob(f"{output_dir}/*.txt")
srt_files = glob.glob(f"{output_dir}/*.srt")

try:
  txt_files.remove('output/archive.txt')
except ValueError:
  pass

for file in txt_files + srt_files:
  files.download(file)

جارٍ تجهيز بيئة العمل.


جارٍ تحويل المواد إلى نصوص باستخدام نماذج Whisper.


URL elements:   0%|          | 0/1 [00:00<?, ?it/s]

0.00%|          | 0.00/2609.37 [00:00<?, ?sec/s]

0.06%|          | 1.62/2609.37 [00:44<19:45:48, 27.28s/sec]

1.27%|▏         | 33.12/2609.37 [01:25<1:35:20,  2.22s/sec]

2.32%|▏         | 60.52/2609.37 [02:13<1:23:25,  1.96s/sec]

3.33%|▎         | 86.92/2609.37 [03:09<1:25:13,  2.03s/sec]

4.53%|▍         | 118.26/2609.37 [03:57<1:15:34,  1.82s/sec]

5.58%|▌         | 145.58/2609.37 [04:44<1:13:16,  1.78s/sec]

6.83%|▋         | 178.30/2609.37 [05:38<1:10:13,  1.73s/sec]

7.83%|▊         | 204.28/2609.37 [05:58<58:06,  1.45s/sec]  

9.09%|▉         | 237.10/2609.37 [06:58<1:02:21,  1.58s/sec]

10.20%|█         | 266.08/2609.37 [07:56<1:06:56,  1.71s/sec]

11.36%|█▏        | 296.44/2609.37 [09:29<1:22:13,  2.13s/sec]

12.22%|█▏        | 318.76/2609.37 [16:31<4:00:13,  6.29s/sec]

13.38%|█▎        | 349.02/2609.37 [17:11<2:56:20,  4.68s/sec]

14.45%|█▍        | 377.02/2609.37 [18:17<2:28:12,  3.98s/sec]

15.52%|█▌        | 405.06/26